Você pode adquirir versões impressas e de e-book do *Think Python 3e* (em inglês) em
[Bookshop.org](https://bookshop.org/a/98697/9781098155438) e
[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325).

Uma versão em língua portuguesa da 3ª edição foi publicada pela editora [Novatec](https://novatec.com.br/livros/pense-em-python-3ed/).

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

Crédito: Fotos baixadas de [Lorem Picsum](https://picsum.photos/), um serviço que fornece imagens e uso temporário.
O nome é uma referência a "lorem ipsum", que é um nome para texto de de uso temporário.

In [ ]:
# Esta célula baixa um arquivo compactado que contém os arquivos que
# usaremos nos exemplos deste capítulo.

download('https://github.com/AllenDowney/ThinkPython/raw/v3/photos.zip');

In [ ]:
# AVISO: Esta célula remove o diretório photos/ se ele já existir.
# Quaisquer arquivos que já estejam no diretório photos/ serão excluídos.

# !rm -rf photos/

In [ ]:
!unzip -o photos.zip

Archive:  photos.zip
  inflating: photos/notes.txt        
  inflating: photos/mar-2023/photo2.jpg  
  inflating: photos/mar-2023/photo1.jpg  
  inflating: photos/jan-2023/photo3.jpg  
  inflating: photos/jan-2023/photo2.jpg  
  inflating: photos/jan-2023/photo1.jpg  
  inflating: photos/feb-2023/photo2.jpg  
  inflating: photos/feb-2023/photo1.jpg  


# Arquivos e bancos de dados

A maioria dos programas que vimos até agora são **efêmeros** no sentido de que são executados por um curto período e produzem saída, mas quando terminam, seus dados desaparecem.
Cada vez que você executa um programa efêmero, ele começa do zero.

Outros programas são **persistentes**: eles são executados por um longo período (ou o tempo todo); eles mantêm pelo menos alguns de seus dados em armazenamento de longo prazo; e se eles encerram e reiniciam, eles continuam de onde pararam.

Uma maneira simples para os programas manterem seus dados é lendo e gravando arquivos de texto.
Uma alternativa mais versátil é armazenar dados em um banco de dados.
Bancos de dados são arquivos especializados que podem ser lidos e gravados de forma mais eficiente do que arquivos de texto, e eles fornecem recursos adicionais.

Neste capítulo, escreveremos programas que leem e gravam arquivos de texto e bancos de dados, e como exercício você escreverá um programa que pesquisa uma coleção de fotos em busca de duplicatas.
Mas antes de poder trabalhar com um arquivo, você precisa encontrá-lo, então começaremos com nomes de arquivos, caminhos e diretórios.

## Nomes de arquivos e caminhos

Os arquivos são organizados em **diretórios**, também chamados de "pastas".
Todo programa em execução tem um **diretório de trabalho atual**, que é o diretório padrão para a maioria das operações.
Por exemplo, quando você abre um arquivo, o Python o procura no diretório de trabalho atual.

O módulo `os` fornece funções para trabalhar com arquivos e diretórios ("os" significa "sistema operacional" do inglês *operating system*).
Ele fornece uma função chamada `getcwd` que obtém o nome do diretório de trabalho atual.

In [ ]:
# esta célula substitui `os.cwd` por uma função que devolve um caminho falso

import os

def getcwd():
    return "/home/dinsdale"

os.getcwd = getcwd

In [ ]:
import os

os.getcwd()

'/home/dinsdale'

O resultado neste exemplo é o diretório *home* de um usuário chamado `dinsdale`.
Uma *string* como `'/home/dinsdale'` que identifica um arquivo ou diretório é chamada de **caminho**.

Um nome de arquivo simples como `'memo.txt'` também é considerado um caminho, mas é um **caminho relativo** porque especifica um nome de arquivo relativo ao diretório atual.
Neste exemplo, o diretório atual é `'/home/dinsdale'`, então `'memo.txt'` é equivalente ao caminho completo `'/home/dinsdale/memo.txt'`.

Um caminho que começa com `'/'` não depende do diretório atual -- é chamado de **caminho absoluto**.
Para encontrar o caminho absoluto para um arquivo, você pode usar `abspath`:

In [ ]:
os.path.abspath('memo.txt')

'/home/dinsdale/memo.txt'

O módulo `os` possui outras funções para trabalhar com nomes de arquivos e caminhos.
`listdir` devolve uma lista do conteúdo do diretório fornecido, incluindo arquivos e outros diretórios.
Aqui está um exemplo que lista o conteúdo de um diretório chamado `photos`:

In [ ]:
os.listdir('photos')

['digests.dat',
 'digests.dir',
 'notes.txt',
 'new_notes.txt',
 'mar-2023',
 'digests.bak',
 'jan-2023',
 'feb-2023']

Este diretório contém um arquivo de texto chamado `notes.txt` e três diretórios.
Os diretórios contêm arquivos de imagem no formato JPEG:

In [ ]:
os.listdir('photos/jan-2023')

['photo3.jpg', 'photo2.jpg', 'photo1.jpg']

Para verificar se um arquivo ou diretório existe, podemos usar `os.path.exists`:

In [ ]:
os.path.exists('photos')

True

In [ ]:
os.path.exists('photos/apr-2023')

False

Para verificar se um caminho se refere a um arquivo ou diretório, podemos usar `isdir`, que devolve `True` se um caminho se refere a um diretório.

In [ ]:
os.path.isdir('photos')

True

E `isfile` que devolve `True` se um caminho se refere a um arquivo.

In [ ]:
os.path.isfile('photos/notes.txt')

True

Um desafio de trabalhar com caminhos é que eles são formatados em formas diferentes em sistemas operacionais diferentes.
Em sistemas macOS e UNIX como Linux, os nomes de diretórios e arquivos em um caminho são separados por uma barra, `/`.
O Windows usa uma barra invertida, `\`.
Então, se você executar esses exemplos no Windows, verá barras invertidas nos caminhos e terá que substituir as barras nos exemplos.

Ou, para escrever código que funcione em ambos os sistemas, você pode usar `os.path.join`, que une diretórios e nomes de arquivo em um caminho usando uma barra ou uma barra invertida, dependendo do sistema operacional que você estiver usando:

In [ ]:
os.path.join('photos', 'jan-2023', 'photo1.jpg')

'photos/jan-2023/photo1.jpg'

Mais adiante neste capítulo, usaremos essas funções para pesquisar um conjunto de diretórios e encontrar todos os arquivos de imagem.

## *f-strings*

Uma maneira de programas armazenarem dados é gravá-los em um arquivo de texto.
Por exemplo, suponha que você seja um observador de camelos e queira registrar o número de camelos que viu durante um período de observação.
E suponha que em um ano e meio, você tenha avistado `23` camelos.
Os dados em seu livro de observação de camelos podem ser assim:

In [ ]:
num_years = 1.5
num_camels = 23

Para gravar esses dados em um arquivo, você pode usar o método `write`, que vimos no Capítulo 8.
O argumento de `write` tem que ser uma *string*, então se quisermos colocar outros valores em um arquivo, temos que convertê-los em *strings*.
A maneira mais fácil de fazer isso é com a função interna `str`.

É assim que fica:

In [ ]:
writer = open('camel-spotting-book.txt', 'w')
writer.write(str(num_years))
writer.write(str(num_camels))
writer.close()

That works, but `write` doesn't add a space or newline unless you include it explicitly.
If we read back the file, we see that the two numbers are run together.

Isso funciona, mas `write` não adiciona um espaço ou nova linha a menos que você os inclua explicitamente.
Se lermos o arquivo novamente, veremos que os dois números são escritos juntos:

In [ ]:
open('camel-spotting-book.txt').read()

'1.523'

No mínimo, devemos adicionar espaços em branco entre os números.
E enquanto estamos nisso, vamos adicionar algum texto explicativo.

Para escrever uma combinação de *strings* e outros valores, podemos usar uma **f-string**, que é uma *string* que tem a letra `f` antes da aspas de abertura e contém uma ou mais expressões Python entre chaves.
A seguinte f-string contém uma expressão, que é um nome de variável:

In [ ]:
f'I have spotted {num_camels} camels'

'I have spotted 23 camels'

O resultado é uma *string* em que a expressão foi avaliada e substituída pelo resultado.
Pode haver mais de uma expressão:

In [ ]:
f'In {num_years} years I have spotted {num_camels} camels'

'In 1.5 years I have spotted 23 camels'

E as expressões podem conter operadores e chamadas de função:

In [ ]:
line = f'In {round(num_years * 12)} months I have spotted {num_camels} camels'
line

'In 18 months I have spotted 23 camels'

Então poderíamos gravar os dados em um arquivo de texto como este:

In [ ]:
writer = open('camel-spotting-book.txt', 'w')
writer.write(f'Years of observation: {num_years}\n')
writer.write(f'Camels spotted: {num_camels}\n')
writer.close()

Ambas as *f-strings* terminam com a sequência `\n`, que adiciona um caractere de nova linha.

Podemos ler o arquivo de novo assim:

In [ ]:
data = open('camel-spotting-book.txt').read()
print(data)

Years of observation: 1.5
Camels spotted: 23



Em uma *f-string*, uma expressão entre chaves é convertida em uma *string*, assim que você possa incluir listas, dicionários e outros tipos.

In [ ]:
t = [1, 2, 3]
d = {'one': 1}
f'Here is a list {t} and a dictionary {d}'

"Here is a list [1, 2, 3] and a dictionary {'one': 1}"

Se uma *f-string* contiver uma expressão inválida, o resultado será um erro:

In [ ]:
%%expect TypeError

f'This is not a valid expression {t + 2}'

TypeError: can only concatenate list (not "int") to list

## YAML

Um dos motivos pelos quais os programas leem e gravam arquivos é para armazenar **dados de configuração**, que são informações que especificam o que o programa deve fazer e como.

Por exemplo, em um programa que procura fotos duplicadas, podemos ter um dicionário chamado `config` que contém o nome do diretório a ser pesquisado, o nome de outro diretório onde ele deve armazenar os resultados e uma lista de extensões de arquivo que ele deve usar para identificar arquivos de imagem.

Aqui está como pode ser:

In [ ]:
config = {
    'photo_dir': 'photos',
    'data_dir': 'photo_info',
    'extensions': ['jpg', 'jpeg'],
}

Para escrever esses dados em um arquivo de texto, poderíamos usar *f-strings*, como na seção anterior. Mas é mais fácil usar um módulo chamado `yaml` que é projetado para esse tipo de coisa.

O módulo `yaml` fornece funções para trabalhar com arquivos YAML, que são arquivos de texto formatados para serem fáceis para humanos *e* programas lerem e escreverem.

Aqui está um exemplo que usa a função `dump` para escrever o dicionário `config` em um arquivo YAML:

In [ ]:
# esta célula instala o pacote pyyaml, que fornece o módulo yaml

try:
    import yaml
except ImportError:
    !pip install pyyaml

In [ ]:
import yaml

config_filename = 'config.yaml'
writer = open(config_filename, 'w')
yaml.dump(config, writer)
writer.close()

Se lermos novamente o conteúdo do arquivo, podemos ver como é o formato YAML:

In [ ]:
readback = open(config_filename).read()
print(readback)

data_dir: photo_info
extensions:
- jpg
- jpeg
photo_dir: photos



Agora, podemos usar `safe_load` para ler o arquivo YAML:

In [ ]:
reader = open(config_filename)
config_readback = yaml.safe_load(reader)
config_readback

{'data_dir': 'photo_info',
 'extensions': ['jpg', 'jpeg'],
 'photo_dir': 'photos'}

O resultado é um novo dicionário que contém as mesmas informações do original, mas não é o mesmo dicionário:

In [ ]:
config is config_readback

False

Converter um objeto como um dicionário em uma *string* é chamado de **serialização**.
Converter a *string* de volta em um objeto é chamado de **desserialização**.
Se você serializar e depois desserializar um objeto, o resultado deve ser equivalente ao original.

## *Shelve*

Até agora, lemos e escrevemos arquivos de texto -- agora vamos considerar bancos de dados.
Um **banco de dados** é um arquivo organizado para armazenar dados.
Alguns bancos de dados são organizados como uma tabela com linhas e colunas de informações.
Outros são organizados como um dicionário que mapeia de chaves para valores; às vezes são chamados de **armazenamentos de chave-valor**.

O módulo `shelve` possui funções para criar e atualizar um armazenamento de chave-valor chamado *prateleira* (em inglês, "*shelf*").
Como exemplo, criaremos uma prateleira para conter legendas para as figuras no diretório `photos`.
Usaremos o dicionário `config` para obter o nome do diretório onde devemos colocar a prateleira:

In [ ]:
haconfig['data_dir']

'photo_info'

Podemos usar `os.makedirs` para criar este diretório, caso ele ainda não exista:

In [ ]:
os.makedirs(config['data_dir'], exist_ok=True)

E `os.path.join` para criar um caminho que inclui o nome do diretório e o nome do arquivo *shelf*, `captions`:

In [ ]:
db_file = os.path.join(config['data_dir'], 'captions')
db_file

'photo_info/captions'

Agora podemos usar `shelve.open` para abrir o arquivo prateleira.
O argumento `c` indica que o arquivo deve ser criado se necessário:

In [ ]:
import shelve

db = shelve.open(db_file, 'c')
db

O valor devolvido é oficialmente um objeto `DbfilenameShelf`, mas frequente chamado apenas de objeto prateleira.

O objeto prateleira muitas vezes se comporta como um dicionário.
Por exemplo, podemos usar o operador colchetes para adicionar um item, que é um mapeamento de uma chave para um valor:

In [ ]:
key = 'jan-2023/photo1.jpg'
db[key] = 'Cat nose'

Neste exemplo, a chave é o caminho para um arquivo de imagem e o valor é uma *string* que descreve a imagem.

Também usamos o operador de colchetes para a partir de uma chave obter o valor correspondente:

In [ ]:
value = db[key]
value

'Cat nose'

Se você fizer outra atribuição a uma chave existente, `shelve` substituirá o valor antigo:

In [ ]:
db[key] = 'Close up view of a cat nose'
db[key]

'Close up view of a cat nose'

Alguns métodos de dicionário, como `keys`, `values` e `items`, também funcionam com objetos prateleira:

In [ ]:
list(db.keys())

['jan-2023/photo1.jpg']

In [ ]:
list(db.values())

['Close up view of a cat nose']

Podemos usar o operador `in` para verificar se uma chave aparece na prateleira:

In [ ]:
key in db

True

E podemos usar uma instrução `for` para percorrer as chaves:

In [ ]:
for key in db:
    print(key, ':', db[key])

jan-2023/photo1.jpg : Close up view of a cat nose


Assim como acontece com outros arquivos, você deve fechar o banco de dados quando terminar:

In [ ]:
db.close()

Agora, se listarmos o conteúdo do diretório de dados, veremos dois arquivos:

In [ ]:
# Quando você abre um arquivo prateleira, um arquivo de backup é criado com o
# sufixo `.bak`.
# Se você executar este notebook mais de uma vez, poderá ver que esse arquivo
# foi criado.
# Esta célula o remove, então a saída mostrada no livro está correta.

!rm -f photo_info/captions.bak

In [ ]:
os.listdir(config['data_dir'])

['captions.dir', 'captions.dat']

`captions.dat` contém os dados que acabamos de armazenar.
`captions.dir` contém informações sobre a organização do banco de dados que o torna mais eficiente para acessar.
O sufixo `dir` significa "diretório", mas não tem nada a ver com os diretórios com os quais temos trabalhado que contêm arquivos.

## Armazenando estruturas de dados

No exemplo anterior, as chaves e valores na prateleira são *strings*.
Mas também podemos usar uma prateleira para conter estruturas de dados como listas e dicionários.

Como exemplo, vamos revisitar o exemplo do anagrama de um exercício no [Capítulo 11](https://colab.research.google.com/github/rodrigocarlson/PensePython3ed/blob/main/capitulos/chap11.ipynb).
Lembre-se de que fizemos um dicionário que mapeia uma sequência de letras ordenadas para a lista de palavras que podem ser escritas com essas letras.
Por exemplo, a chave `'opst'` mapeia para a lista `['opts', 'post', 'pots', 'spot', 'stop', 'tops']`.

Usaremos a seguinte função para classificar as letras em uma palavra:

In [ ]:
def sort_word(word):
    return ''.join(sorted(word))

E aqui está um exemplo:

In [ ]:
word = 'pots'
key = sort_word(word)
key

'opst'

Agora vamos abrir uma prateleira chamada `anagram_map`.
O argumento `'n'` significa que devemos sempre criar uma nova prateleira vazia, mesmo que já exista uma:

In [ ]:
db = shelve.open('anagram_map', 'n')

Agora podemos adicionar um item à prateleira assim:

In [ ]:
db[key] = [word]
db[key]

['pots']

Neste item, a chave é uma *string* e o valor é uma lista de *strings*.

Agora suponha que encontramos outra palavra que contém as mesmas letras, como `tops`:

In [ ]:
word = 'tops'
key = sort_word(word)
key

'opst'

A chave é a mesma do exemplo anterior, então queremos acrescentar uma segunda palavra à mesma lista de *strings*.
Veja como faríamos se `db` fosse um dicionário.

In [ ]:
db[key].append(word)          # INCORRETO

Mas se executarmos isso e recuperarmos o valor associado à chave na prateleira, parece que ela não foi atualizada:

In [ ]:
db[key]

['pots']

Aqui está o problema: quando usamos a chave, obtemos uma lista de *strings*, mas se modificarmos a lista de *strings*, isso não afeta a prateleira.
Se quisermos atualizar a prateleira, temos que ler o valor antigo, atualizá-lo e, então, escrever o novo valor de volta na prateleira:

In [ ]:
anagram_list = db[key]
anagram_list.append(word)
db[key] = anagram_list

Agora o valor na prateleira está atualizado:

In [ ]:
db[key]

['pots', 'tops']

Como exercício, você pode terminar este exemplo lendo a lista de palavras e armazenando todos os anagramas em uma prateleira.

In [ ]:
db.close()

## Verificando arquivos equivalentes

Se os arquivos contiverem imagens, temos que abri-los com o modo `'rb'`, em que `'r'` significa que queremos ler o conteúdo e `'b'` indica **modo binário**.
No modo binário, o conteúdo não é interpretado como texto -- ele é tratado como uma sequência de bytes.

Aqui está um exemplo que abre e lê um arquivo de imagem:

In [ ]:
path1 = 'photos/jan-2023/photo1.jpg'
data1 = open(path1, 'rb').read()
type(data1)

bytes

O resultado de `read` é um objeto `bytes` -- como o nome sugere, ele contém uma sequência de bytes.

Em geral, o conteúdo de um arquivo de imagem não é legível por humanos.
Mas se lermos o conteúdo de um segundo arquivo, podemos usar o operador `==` para comparar:

In [ ]:
path2 = 'photos/jan-2023/photo2.jpg'
data2 = open(path2, 'rb').read()
data1 == data2

False

Esses dois arquivos não são equivalentes.

Vamos encapsular o que temos até agora em uma função:

In [ ]:
def same_contents(path1, path2):
    data1 = open(path1, 'rb').read()
    data2 = open(path2, 'rb').read()
    return data1 == data2

Se tivermos apenas dois arquivos, esta função é uma boa opção.
Mas suponha que temos um grande número de arquivos e queremos saber se quaisquer dois deles contêm os mesmos dados.
Seria ineficiente comparar cada par de arquivos.

Uma alternativa é usar uma **função hash**, que recebe o conteúdo de um arquivo e calcula um ***digest***, que geralmente é um inteiro grande.
Se dois arquivos contiverem os mesmos dados, eles terão o mesmo *digest*.
Se dois arquivos forem diferentes, eles *quase sempre* terão *digests* diferentes.

O módulo `hashlib` fornece várias funções hash -- a que usaremos é chamada `md5`.
Começaremos usando `hashlib.md5` para criar um objeto `HASH`:

In [ ]:
import hashlib

md5_hash = hashlib.md5()
type(md5_hash)

_hashlib.HASH

O objeto `HASH` possui um método `update` que recebe o conteúdo do arquivo como argumento:

In [ ]:
md5_hash.update(data1)

Agora podemos usar `hexdigest` para obter o *digest* como uma sequência de dígitos hexadecimais que representam um inteiro na base 16.

In [ ]:
digest = md5_hash.hexdigest()
digest

'aa1d2fc25b7ae247b2931f5a0882fa37'

A função a seguir encapsula essas etapas:

In [ ]:
def md5_digest(filename):
    data = open(filename, 'rb').read()
    md5_hash = hashlib.md5()
    md5_hash.update(data)
    digest = md5_hash.hexdigest()
    return digest

Se fizermos *hash* do conteúdo de um arquivo diferente, podemos confirmar que obtemos um *digest* diferente:

In [ ]:
filename2 = 'photos/feb-2023/photo2.jpg'
md5_digest(filename2)

'6a501b11b01f89af9c3f6591d7f02c49'

Agora temos quase tudo o que precisamos para encontrar arquivos equivalentes.
O último passo é pesquisar um diretório e encontrar todos os arquivos de imagens.

## Percorrendo diretórios

A função a seguir recebe como argumento o diretório que queremos pesquisar.
Ela usa `listdir` para percorrer o conteúdo do diretório.
Quando encontra um arquivo, ela exibe seu caminho completo.
Quando encontra um diretório, ela se chama recursivamente para pesquisar o subdiretório.

In [ ]:
def walk(dirname):
    for name in os.listdir(dirname):
        path = os.path.join(dirname, name)

        if os.path.isfile(path):
            print(path)
        elif os.path.isdir(path):
            walk(path)

Podemos usá-la assim:

In [ ]:
walk('photos')

photos/digests.dat
photos/digests.dir
photos/notes.txt
photos/new_notes.txt
photos/mar-2023/photo2.jpg
photos/mar-2023/photo1.jpg
photos/digests.bak
photos/jan-2023/photo3.jpg
photos/jan-2023/photo2.jpg
photos/jan-2023/photo1.jpg
photos/feb-2023/photo2.jpg
photos/feb-2023/photo1.jpg


A ordem dos resultados depende dos detalhes do sistema operacional.

## Depuração

Ao ler e escrever arquivos, você pode ter problemas com espaços em branco.
Esses erros podem ser difíceis de depurar porque os caracteres de espaço em branco normalmente são invisíveis.
Por exemplo, aqui está uma *string* que contém espaços, uma tabulação representada pela sequência `\t` e uma nova linha representada pela sequência `\n`.
Quando a exibimos, não vemos os caracteres de espaço em branco:

In [ ]:
s = '1 2\t 3\n 4'
print(s)

1 2	 3
 4


A função interna `repr` pode ajudar. Ela recebe qualquer objeto como argumento e devolve uma representação de string do objeto.
Para strings, ela representa caracteres de espaço em branco com sequências de barras invertidas:

In [ ]:
print(repr(s))

'1 2\t 3\n 4'


Isso pode ser útil para depuração.

Outro problema que você pode encontrar é que sistemas diferentes usam caracteres diferentes para indicar o fim de uma linha. Alguns sistemas usam uma nova linha, representada por `\n`. Outros usam um caractere de retorno, representado por `\r`.
Alguns usam ambos. Se você mover arquivos entre sistemas diferentes, essas
inconsistências podem causar problemas.

A capitalização do nome do arquivo é outro problema que você pode encontrar se trabalhar com sistemas operacionais diferentes.
No macOS e UNIX, os nomes de arquivo podem conter letras maiúsculas e minúsculas, dígitos e a maioria dos símbolos.
Mas muitos aplicativos do Windows ignoram a diferença entre letras maiúsculas e minúsculas, e vários símbolos que são permitidos no macOS e UNIX não são permitidos no Windows.

## Glossário

**efêmero** (*ephemeral*)**:**
Um programa efêmero normalmente é executado por um curto período e, quando encerra, seus dados são perdidos.

**persistente** (*persistent*)**:**
Um programa persistente é executado indefinidamente e mantém pelo menos alguns de seus dados em armazenamento permanente.

**diretório** (*directory*)**:**
Uma coleção de arquivos e outros diretórios.

**diretório de trabalho atual** (*current working directory*)**:**
O diretório padrão usado por um programa, a menos que outro diretório seja especificado.

**caminho** (*path*)**:**
Uma *string* que especifica uma sequência de diretórios, geralmente levando a um arquivo.

**caminho relativo** (*relative path*)**:**
Um caminho que começa no diretório de trabalho atual ou em algum outro diretório especificado.

**caminho absoluto** (*absolute path*)**:**
Um caminho que não depende do diretório atual.

***f-string*:**
Uma sequência de caracteres que tem a letra `f` antes das aspas de abertura e contém uma ou mais expressões entre chaves.

**dados de configuração** (*configuration data*)**:**
Dados, geralmente armazenados em um arquivo, que especificam o que um programa deve fazer e como.

**serialização** (*serialization*)**:**
Conversão de um objeto em uma *string*.

**desserialização** (*deserialization*)**:**
Conversão de uma *string* em um objeto.

**banco de dados** (*database*)**:**
Um arquivo cujo conteúdo é organizado para executar certas operações de forma eficiente.

**armazenamentos de chave-valor** (*key-value stores*)**:**
Um banco de dados cujo conteúdo é organizado como um dicionário com chaves que correspondem a valores.

**modo binário** (*binary mode*)**:**
Uma maneira de abrir um arquivo para que o conteúdo seja interpretado como uma sequência de bytes em vez de uma sequência de caracteres.

**função *hash*** (*hash function*)**:**
Uma função que recebe um objeto e calcula um inteiro, que às vezes é chamado de *digest*.

***digest*:**
O resultado de uma função *hash*, especialmente quando é usada para verificar se dois objetos são iguais.

## Exercícios

In [ ]:
# Esta célula diz ao Jupyter para fornecer informações detalhadas de depuração
# quando ocorre um erro de tempo de execução. Execute-a antes de trabalhar nos
# exercícios.

%xmode Verbose

Exception reporting mode: Verbose


### Pergunte a um assistente virtual

Há vários tópicos que surgiram neste capítulo que não expliquei em detalhes.
Aqui estão algumas perguntas que você pode fazer a um assistente virtual para obter mais informações.

* "Quais são as diferenças entre programas efêmeros e persistentes?" ("*What are the differences between ephemeral and persistent programs?*")

* "Quais são alguns exemplos de programas persistentes?" ("*What are some examples of persistent programs?*")

* "Qual é a diferença entre um caminho relativo e um caminho absoluto?" ("*What's the difference between a relative path and an absolute path?*")

* "Por que o módulo `yaml` tem funções chamadas `load` e `safe_load`?" ("*Why does the `yaml` module have functions called `load` and `safe_load`?*)"

* "Quando escrevo uma prateleira Python, quais são os arquivos com sufixos `dat` e `dir`?" ("*When I write a Python shelf, what are the files with suffixes `dat` and `dir`?*")

* "Além dos armazenamentos de chave-valor, que outros tipos de bancos de dados existem?" ("*Other than key-values stores, what other kinds of databases are there?*")

* "Quando leio um arquivo, qual é a diferença entre o modo binário e o modo texto?" ("*When I read a file, what's the difference between binary mode and text mode?*")

* "Quais são as diferenças entre um objeto *bytes* e uma *string*?" ("*What are the differences between a bytes object and a string?*")

* "O que é uma função *hash*?" ("*What is a hash function?*")

* "O que é um *digest* MD5?" ("*What is an MD5 digest?*")

Como sempre, se você ficar preso em algum dos exercícios a seguir, considere pedir ajuda a um assistente virtual. Junto com sua pergunta, você pode querer colar as funções relevantes deste capítulo.

### Exercício

Escreva uma função chamada `replace_all` que recebe como argumentos uma *string* padrão, uma *string* de substituição e dois nomes de arquivo.
Ela deve ler o primeiro arquivo e gravar o conteúdo no segundo arquivo (criando-o se necessário).
Se a *string* padrão aparecer em qualquer lugar do conteúdo, ela deve ser substituída pela *string* de substituição.

Aqui está um esboço da função para você começar:

In [ ]:
def replace_all(old, new, source_path, dest_path):
    # lê o conteúdo do arquivo de origem
    reader = open(source_path)

    # substitui a string antiga pela nova

    # escreve o resultado no arquivo de destino

In [ ]:
# Solução

def replace_all(old, new, source_path, dest_path):
    reader = open(source_path)
    contents = reader.read()
    reader.close()

    contents = contents.replace(old, new)

    writer = open(dest_path, 'w')
    writer.write(contents)
    writer.close()

Para testar sua função, leia o arquivo `photos/notes.txt`, substitua `'photos'` por `'images'` e grave o resultado no arquivo `photos/new_notes.txt`:

In [ ]:
source_path = 'photos/notes.txt'
open(source_path).read()

'These photos are from Lorem Picsum at https://picsum.photos\n'

In [ ]:
dest_path = 'photos/new_notes.txt'
old = 'photos'
new = 'images'
replace_all(old, new, source_path, dest_path)

In [ ]:
open(dest_path).read()

'These images are from Lorem Picsum at https://picsum.images\n'

### Exercício

Em [uma seção anterior](section_storing_data_structure), usamos o módulo `shelve` para fazer um armazenamento de chave-valor que mapeia uma sequência de letras ordenada para uma lista de anagramas.
Para finalizar o exemplo, escreva uma função chamada `add_word` que recebe como argumentos uma sequência e um objeto shelf.

Ela deve ordenar as letras da palavra para fazer uma chave e, em seguida, verificar se a chave já está na prateleira. Se não, ela deve fazer uma lista que contenha a nova palavra e adicioná-la à prateleira. Se sim, ela deve anexar a nova palavra ao valor existente.

In [ ]:
# Solução

def add_word(word, db):
    key = sort_word(word)

    if key not in db:
        db[key] = [word]
    else:
        anagrams = db[key]
        anagrams.append(word)
        db[key] = anagrams

Você pode usar este laço de repetição para testar sua função:

In [ ]:
download('https://raw.githubusercontent.com/AllenDowney/ThinkPython/v3/words.txt');

In [ ]:
word_list = open('words.txt').read().split()

db = shelve.open('anagram_map', 'n')
for word in word_list:
    add_word(word, db)

Se tudo estiver funcionando, você conseguirá usar uma chave como `'opst'` e obter uma lista de palavras que podem ser escritas com essas letras:

In [ ]:
db['opst']

['opts', 'post', 'pots', 'spot', 'stop', 'tops']

In [ ]:
for key, value in db.items():
    if len(value) > 8:
        print(value)

['alerts', 'alters', 'artels', 'estral', 'laster', 'ratels', 'salter', 'slater', 'staler', 'stelar', 'talers']
['apers', 'asper', 'pares', 'parse', 'pears', 'prase', 'presa', 'rapes', 'reaps', 'spare', 'spear']
['capers', 'crapes', 'escarp', 'pacers', 'parsec', 'recaps', 'scrape', 'secpar', 'spacer']
['estrin', 'inerts', 'insert', 'inters', 'niters', 'nitres', 'sinter', 'triens', 'trines']
['least', 'setal', 'slate', 'stale', 'steal', 'stela', 'taels', 'tales', 'teals', 'tesla']


In [ ]:
db.close()

### Exercício

Em uma grande coleção de arquivos, pode haver mais de uma cópia do mesmo arquivo, armazenada em diretórios diferentes ou com nomes de arquivo diferentes.
O objetivo deste exercício é procurar por duplicatas.
Como exemplo, trabalharemos com arquivos de imagem no diretório `photos`.

Veja como funcionará:

* Usaremos a função `walk` de [](section_walking_directories) para procurar neste diretório por arquivos que terminam com uma das extensões em `config['extensions']`.

* Para cada arquivo, usaremos `md5_digest` de [](section_md5_digest) para computar um *digest* do conteúdo.

* Usando uma prateleira, faremos um mapeamento de cada *digest* para uma lista de caminhos com esse *digest*.

* Finalmente, procuraremos na prateleira por quaisquer *digests* que mapeiem para vários arquivos.

* Se encontrarmos algum, usaremos `same_contents` para confirmar que os arquivos contêm os mesmos dados.

Vou sugerir algumas funções para escrever primeiro, depois vamos juntar tudo.

1. Para identificar arquivos de imagem, escreva uma função chamada `is_image` que recebe um caminho e uma lista de extensões de arquivo e devolve `True` se o caminho termina com uma das extensões na lista. Dica: Use `os.path.splitext` -- ou peça para um assistente virtual escrever esta função para você.

In [ ]:
# Solução

def is_image(path, extensions):
    """Verifica se o caminho termina com um das extensões.

    path: string como o caminho do arquivo
    extensions: lista de extensões

    >>> is_image('photo.jpg', ['jpg', 'jpeg'])
    True
    >>> is_image('PHOTO.JPG', ['jpg', 'jpeg'])
    True
    >>> is_image('notes.txt', ['jpg', 'jpeg'])
    False
    """
    _, extension = os.path.splitext(path)
    return extension.strip('.').lower() in extensions

Você pode usar `doctest` para testar sua função:

In [ ]:
from doctest import run_docstring_examples

def run_doctests(func):
    run_docstring_examples(func, globals(), name=func.__name__)

run_doctests(is_image)

2. Escreva uma função chamada `add_path` que recebe como argumentos um caminho e uma prateleira. Ela deve usar `md5_digest` para computar um *digest* do conteúdo do arquivo. Então, ela deve atualizar a prateleira, criando um novo item que mapeia do *dugest* para uma lista contendo o caminho, ou anexando o caminho à lista, se existir.

In [ ]:
# Solução

def add_path(path, db):
    digest = md5_digest(path)

    if digest not in db:
        paths = [path]
    else:
        paths = db[digest]
        paths.append(path)

    db[digest] = paths

3. Escreva uma versão de `walk` chamada `walk_images` que recebe um diretório e percorre os arquivos no diretório e seus subdiretórios. Para cada arquivo, ele deve usar `is_image` para verificar se é um arquivo de imagem e `add_path` para adicioná-lo à prateleira.

In [ ]:
# Solução

def walk_images(dirname):
    for name in os.listdir(dirname):
        path = os.path.join(dirname, name)

        if os.path.isfile(path):
            if is_image(path, config['extensions']):
                add_path(path, db)
        else:
            walk_images(path)

Quando tudo estiver funcionando, você pode usar o programa a seguir para criar a prateleira, pesquisar no diretório `photos` e adicionar caminhos para a prateleira e, então, verificar se há vários arquivos com o mesmo *digest*.

In [ ]:
db = shelve.open('photos/digests', 'n')
walk_images('photos')

for digest, paths in db.items():
    if len(paths) > 1:
        print(paths)

['photos/mar-2023/photo2.jpg', 'photos/jan-2023/photo1.jpg']


Você deveria encontrar um par de arquivos que têm o mesmo *digest*.
Use `same_contents` para verificar se eles contêm os mesmos dados.

In [ ]:
# Solução

path1, path2 = ['photos/mar-2023/photo2.jpg', 'photos/jan-2023/photo1.jpg']
same_contents(path1, path2)

True

[Pense Python: 3ª Edição](https://rodrigocarlson.github.io/PensePython3ed/)

Copyright 2024 [Allen B. Downey](https://allendowney.com/) (versão original)

Copyright 2025 [Rodrigo Castelan Carlson](https://rodrigocarlson.paginas.ufsc.br/) (desta versão)

Foram preservadas as mesmas licenças da versão original.

Licença dos códigos: [MIT License](https://mit-license.org/)

Licença dos textos: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)